In [2]:
import spacy
import pandas as pd
from collections import Counter
import re
import os

In [3]:
# Carrega o modelo de NLP em português
nlp = spacy.load("pt_core_news_lg")

In [4]:
# Listas auxiliares para validação
siglas_estados = {"AC", "AL", "AP", "AM", "BA", "CE", "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RS", "RO", "RR", "SC", "SP", "SE", "TO"}
organizacoes_keywords = ["Tribunal", "Ministério", "Banco", "Procuradoria", "Universidade", "Secretaria", "Assembleia"]


In [ ]:
def extract_text_from_txt(txt_path):
    """Lê o conteúdo de um arquivo TXT."""
    try:
        with open(txt_path, "r", encoding="utf-8") as file:
            text = file.read()
        text = text.replace('\n', ' ').replace('\r', ' ')  # Remove quebras de linha
        return text
    except Exception as e:
        print(f"Erro ao ler o arquivo: {e}")
        return ""

In [6]:
def extract_title(text):
    """Extrai uma frase representativa do início do texto como título."""
    doc = nlp(text)
    for sent in doc.sents:
        return sent.text.strip()
    return "Título Desconhecido"

In [8]:
def extract_keywords(text, top_n=10):
    """Extrai palavras-chave do texto baseando-se na frequência e relevância."""
    doc = nlp(text)
    palavras_relevantes = [token.text.lower() for token in doc if token.is_alpha and token.pos_ in ["NOUN", "PROPN", "ADJ"]]
    palavras_mais_frequentes = Counter(palavras_relevantes).most_common(top_n)
    return palavras_mais_frequentes

In [9]:
def limpar_entidades(lista, tipo):
    """Remove entidades curtas, irrelevantes ou que parecem ruído com técnicas heurísticas."""
    resultado = []
    for item in lista:
        item_limpo = item.strip().replace("\n", " ").replace("\r", " ")
        if len(item_limpo) < 3:
            continue
        if tipo == "PESSOA":
            if len(item_limpo.split()) < 2:
                continue
            if not all(p[0].isupper() for p in item_limpo.split()):
                continue
        elif tipo == "ORGANIZACAO":
            if len(item_limpo.split()) < 2 and not any(kw.lower() in item_limpo.lower() for kw in organizacoes_keywords):
                continue
        elif tipo == "LOCAL":
            if item_limpo.upper() not in siglas_estados and len(item_limpo.split()) < 2:
                continue
        elif tipo == "LEI":
            if item_limpo.lower().strip() == "lei":
                continue
        resultado.append(item_limpo)
    return resultado

In [10]:
def analyze_text(text):
    doc = nlp(text)
    titulo = extract_title(text)
    palavras_chave = extract_keywords(text)

    num_palavras = len([token.text for token in doc if token.is_alpha])
    num_caracteres = len(text)
    num_sentencas = len(list(doc.sents))
    num_palavras_unicas = len(set(token.text.lower() for token in doc if token.is_alpha))
    densidade_lexical = num_palavras_unicas / num_palavras if num_palavras > 0 else 0

    contagem_classes = {
        "Verbos": len([token for token in doc if token.pos_ == "VERB"]),
        "Substantivos": len([token for token in doc if token.pos_ == "NOUN"]),
        "Adjetivos": len([token for token in doc if token.pos_ == "ADJ"]),
        "Pronomes": len([token for token in doc if token.pos_ == "PRON"]),
        "Numerais": len([token for token in doc if token.pos_ == "NUM"]),
    }

    entidades = {"PESSOA": [], "ORGANIZACAO": [], "LOCAL": [], "DATA": [], "MONEY": [], "LEI": []}
    entidades_encontradas = []
    for ent in doc.ents:
        label = ent.label_
        if label == "PER":
            entidades["PESSOA"].append(ent.text)
        elif label == "ORG":
            entidades["ORGANIZACAO"].append(ent.text)
        elif label == "LOC":
            entidades["LOCAL"].append(ent.text)
        elif label == "DATE":
            entidades["DATA"].append(ent.text)
        elif label == "MONEY":
            entidades["MONEY"].append(ent.text)
        elif re.search(r"art(igo)?\.?(\s+\d+)?|lei\s+\d+", ent.text.lower()):
            entidades["LEI"].append(ent.text)
        entidades_encontradas.append((ent.text.strip(), label))

    for tipo in ["PESSOA", "ORGANIZACAO", "LOCAL", "LEI"]:
        entidades[tipo] = limpar_entidades(entidades[tipo], tipo)

    df_resumo = pd.DataFrame({
        "Métrica": ["Título do Texto", "Número de Palavras", "Número de Caracteres", "Número de Sentenças", "Palavras Únicas", "Densidade Lexical", "Verbos", "Substantivos", "Adjetivos", "Pronomes", "Numerais"],
        "Valor": [titulo, num_palavras, num_caracteres, num_sentencas, num_palavras_unicas, densidade_lexical,
                  contagem_classes["Verbos"], contagem_classes["Substantivos"], contagem_classes["Adjetivos"],
                  contagem_classes["Pronomes"], contagem_classes["Numerais"]]
    })

    df_palavras_chave = pd.DataFrame(palavras_chave, columns=["Palavra", "Frequência"])
    df_entidades_encontradas = pd.DataFrame(entidades_encontradas, columns=["Entidade", "Tipo"])
    df_entidades = {chave: pd.DataFrame(Counter(valores).items(), columns=["Entidade", "Frequência"]) for chave, valores in entidades.items() if valores}

    return df_resumo, df_entidades, df_entidades_encontradas, df_palavras_chave

In [11]:
def process_folder(folder_path, output_file):
    todos_resumos = []
    todas_entidades_encontradas = []
    todas_palavras_chave = []
    entidades_agrupadas = {}

    for nome_arquivo in os.listdir(folder_path):
        if nome_arquivo.lower().endswith(".txt"):
            caminho_arquivo = os.path.join(folder_path, nome_arquivo)
            print(f"Processando: {caminho_arquivo}")
            texto = extract_text_from_txt(caminho_arquivo)
            if not texto:
                continue
            df_resumo, df_entidades, df_entidades_encontradas, df_palavras_chave = analyze_text(texto)

            df_resumo.insert(0, "Arquivo", nome_arquivo)
            df_entidades_encontradas.insert(0, "Arquivo", nome_arquivo)
            df_palavras_chave.insert(0, "Arquivo", nome_arquivo)

            todos_resumos.append(df_resumo)
            todas_entidades_encontradas.append(df_entidades_encontradas)
            todas_palavras_chave.append(df_palavras_chave)

            for tipo, df in df_entidades.items():
                if tipo not in entidades_agrupadas:
                    entidades_agrupadas[tipo] = []
                df.insert(0, "Arquivo", nome_arquivo)
                entidades_agrupadas[tipo].append(df)

    with pd.ExcelWriter(output_file, engine="xlsxwriter") as writer:
        pd.concat(todos_resumos).to_excel(writer, sheet_name="Resumo", index=False)
        pd.concat(todas_palavras_chave).to_excel(writer, sheet_name="Palavras-chave", index=False)
        pd.concat(todas_entidades_encontradas).to_excel(writer, sheet_name="Entidades Encontradas", index=False)
        for tipo, lista_dfs in entidades_agrupadas.items():
            pd.concat(lista_dfs).to_excel(writer, sheet_name=tipo, index=False)

    print(f"Consolidação concluída! Arquivo salvo em {output_file}")


In [ ]:
# Definindo os caminhos com o formato raw para evitar problemas com as barras invertidas
pasta_textos = r"C:\Users\vitor\OneDrive\Área de Trabalho\TCC\TCC_NLP\j1_diarios_de_justica_stj"
arquivo_saida = r"C:\Users\vitor\OneDrive\Área de Trabalho\TCC\TCC_NLP\resultado_consolidado.xlsx"

process_folder(pasta_textos, arquivo_saida)